<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/GRU_v4_llaj_FE_pilkottu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. Esitiedot ja Tavoite (GRU v4 - Laaj. FE + Soluittain) (2025-04-10 19:44)
"""
Otsoniennuste Helsinki - GRU-malli v4 (Laajennetulla FE:llä ja Vahvistuksilla)

Tavoite:
1. Ladata data, sisältäen pilvisyyden.
2. Suorittaa laajennettu ominaisuusmuokkaus (FE).
3. Määritellä ja kouluttaa GRU-malli käyttäen laajennettuja ominaisuuksia.
4. Arvioida malli ja analysoida tulokset.
5. Visualisoida.
6. Toimitetaan solu kerrallaan selkeyden ja virheiden minimoimiseksi.
"""
print("--- Osa 0: Esitiedot (GRU v4) - OK ---")

In [ ]:
# @title 1. Tuonnit ja Asetukset (Refaktoroitu - GRU) (2025-04-10 19:59) # Päivitetty aika

# Peruskirjastot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os
import math
import copy
import traceback
import warnings
import time # <--- LISÄTTY IMPORT

# Sklearn
try:
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
except ImportError: raise ImportError("scikit-learn puuttuu.")

# PyTorch
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset, DataLoader
except ImportError: raise ImportError("PyTorch puuttuu.")

# Muut
try:
    from tqdm.notebook import tqdm
    import seaborn as sns
except ImportError: tqdm = None; sns = None; print("Tqdm/Seaborn puuttuu.")


# --- Yleisasetukset ---
warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False
print(f"Käytettävä laite: {device}")

# --- Data-asetukset ---
BASE_GITHUB_URL = 'https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/'
PARQUET_PATH = 'data/processed/processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet'
DATA_URL = BASE_GITHUB_URL + PARQUET_PATH
LOCAL_DATA_PATH = 'processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet'
TARGET_COLUMN = 'Otsoni [µg/m³]'
BASE_COLUMNS_TO_LOAD = [
    'Otsoni [µg/m³]', 'Lämpötilan keskiarvo [°C]', 'Keskituulen nopeus [m/s]',
    'Ilmanpaineen keskiarvo [hPa]', 'Tuulen suunnan keskiarvo [°]', 'Pilvisyys [okta]'
]

# --- Ennustus- ja Jakoasetukset ---
FORECAST_HORIZON = 24
SEQUENCE_LENGTH = 72
TEST_SPLIT_RATIO = 0.15
VALID_SPLIT_RATIO = 0.15

# --- RNN/LSTM/GRU Mallin Hyperparametrit ---
RNN_HYPERPARAMS = {
    'model_type': 'GRU',  # Varmistetaan GRU
    'input_size': None,    # Lasketaan myöhemmin
    'hidden_size': 64,
    'num_layers': 2,
    'output_size': FORECAST_HORIZON,
    'dropout_prob': 0.2
}

# --- Neuroverkkojen Koulutusparametrit ---
TRAIN_HYPERPARAMS = {
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 75,
    'patience': 10
}

# --- Arviointiasetukset ---
O3_THRESHOLD_8H_AVG = 85 # µg/m³

# --- Ajanotto ---
script_start_time = time.time() # <--- LISÄTTY AJANOTON ALOITUS

print("\nOsa 1: Tuonnit ja Asetukset (GRU v4) - SUORITETTU ONNISTUNEESTI.") # Päivitetty vahvistusviesti
print(f"Kohdemuuttuja: {TARGET_COLUMN}")
print(f"Käytettävä RNN-tyyppi: {RNN_HYPERPARAMS['model_type']}")
print(f"8h Keskiarvon kynnysarvo: {O3_THRESHOLD_8H_AVG} µg/m³")

In [ ]:
# @title 2. Funktiot Datan Lataukseen ja Käsittelyyn (Laajennettu FE) (2025-04-10 19:44)

import pandas as pd
import numpy as np
import requests
import io
import os
import re
import traceback
# Varmistetaan tuonnit
try: from sklearn.preprocessing import StandardScaler
except ImportError: StandardScaler = None
try: import seaborn as sns; import matplotlib.pyplot as plt
except ImportError: sns = None; plt = None

# --- Funktiot datan lataamiseen ---
def download_data(url, local_path):
    try:
        print(f"Yritetään ladata: {url[:60]}..."); response = requests.get(url); response.raise_for_status()
        target_dir = os.path.dirname(local_path);
        if target_dir and not os.path.exists(target_dir): os.makedirs(target_dir, exist_ok=True)
        with open(local_path, 'wb') as f: f.write(response.content)
        print(f"Ladattu: {local_path}"); return True
    except Exception as e: print(f"Latausvirhe: {e}"); return False

def load_parquet_data(filepath_or_url, local_cache_path="default_cache.parquet"):
    filepath_to_read = None
    if filepath_or_url.startswith('http'):
         if os.path.exists(local_cache_path): filepath_to_read = local_cache_path; print(f"Käytetään välimuistia: {local_cache_path}")
         else: filepath_to_read = local_cache_path if download_data(filepath_or_url, local_cache_path) else None
    else: filepath_to_read = filepath_or_url
    if not filepath_to_read or not os.path.exists(filepath_to_read): print(f"VIRHE: Tiedostoa '{filepath_to_read}' ei löytynyt."); return None
    try:
        print(f"Ladataan: {filepath_to_read}"); df = pd.read_parquet(filepath_to_read); print(f"Parquet ladattu ({df.shape})")
        if not isinstance(df.index, pd.DatetimeIndex): df.index = pd.to_datetime(df.index)
        if df.index.tz is None: print("Asetetaan TZ=Europe/Helsinki..."); df = df.tz_localize('Europe/Helsinki', ambiguous='NaT', nonexistent='NaT')
        nat_rows = df.index.isnull();
        if nat_rows.any(): print(f"Poistetaan {nat_rows.sum()} NaT-riviä..."); df = df[~nat_rows]
        df.sort_index(inplace=True); print(f"Data aikaväliltä: [{df.index.min()} - {df.index.max()}]")
        if df.isnull().any().any(): print("NaN-arvoja. Täytetään..."); df=df.ffill().bfill(); df.dropna(inplace=True)
        print(f"Lopullinen ladattu muoto: {df.shape}"); return df
    except Exception as e: print(f"VIRHE luvussa/käsittelyssä: {e}"); traceback.print_exc(); return None

# --- Laajennettu Feature Engineering funktio ---
def feature_engineer_advanced_gru(df, target_column):
    print("\nSuoritetaan laajennettu FE..."); df_eng = df.copy()
    if not isinstance(df_eng.index, pd.DatetimeIndex): print("VIRHE: Indeksi ei DatetimeIndex."); return None
    try: # Aika
        df_eng['hour'] = df_eng.index.hour; df_eng['dayofweek'] = df_eng.index.dayofweek; df_eng['dayofyear'] = df_eng.index.dayofyear
        # Korjattu: Käytä df_eng.index.dayofyear (ei Series), ja days_in_year (Series)
        days_in_year = df_eng.index.is_leap_year.map({True: 366.0, False: 365.0})
        df_eng['hour_sin']=np.sin(2*np.pi*df_eng['hour']/24.0); df_eng['hour_cos']=np.cos(2*np.pi*df_eng['hour']/24.0)
        df_eng['dayofweek_sin']=np.sin(2*np.pi*df_eng['dayofweek']/7.0); df_eng['dayofweek_cos']=np.cos(2*np.pi*df_eng['dayofweek']/7.0)
        df_eng['dayofyear_sin']=np.sin(2*np.pi*df_eng['dayofyear']/days_in_year); df_eng['dayofyear_cos']=np.cos(2*np.pi*df_eng['dayofyear']/days_in_year)
        df_eng.drop(columns=['hour', 'dayofweek', 'dayofyear'], inplace=True); print("Lisätty sykliset aikaominaisuudet.")
    except Exception as e: print(f"VIRHE aika FE: {e}")
    wind_dir_col_orig = 'Tuulen suunnan keskiarvo [°]' # Tuuli
    if wind_dir_col_orig in df_eng.columns:
        try:
            wind_dir_numeric = pd.to_numeric(df_eng[wind_dir_col_orig], errors='coerce').ffill().bfill()
            if wind_dir_numeric.notna().all():
                 df_eng['wind_dir_sin']=np.sin(np.deg2rad(wind_dir_numeric)); df_eng['wind_dir_cos']=np.cos(np.deg2rad(wind_dir_numeric))
                 df_eng.drop(columns=[wind_dir_col_orig], inplace=True); print("Muunnettu tuulen suunta sykliseksi.")
            else: print(f"VAROITUS: Tuulensuuntia ei voitu muuntaa numeroksi.")
        except Exception as e: print(f"VIRHE tuulensuunta FE: {e}"); df_eng.drop(columns=['wind_dir_sin','wind_dir_cos'], inplace=True, errors='ignore')
    else: print(f"Saraketta '{wind_dir_col_orig}' ei löytynyt.")
    # Viiveet
    lag_config = { target_column: [1, 2, 3, 6, 12, 24, 48, 72, 168], 'Lämpötilan keskiarvo [°C]': [1, 6, 24], 'Pilvisyys [okta]': [1, 6, 24], 'Keskituulen nopeus [m/s]': [1, 6, 24] }
    print("Luodaan viiveitä..."); original_cols = list(df_eng.columns); lag_cols_added = 0
    for col, lags in lag_config.items():
        if col in original_cols:
            for lag in lags: df_eng[f'{col}_lag_{lag}h'] = df_eng[col].shift(lag); lag_cols_added+=1
        else: print(f"VAROITUS: Sarake '{col}' puuttuu viiveitä varten.")
    print(f"Viiveitä lisätty ({lag_cols_added} kpl).")
    # Liukuvat tilastot
    rolling_config = { target_column: [3, 6, 12, 24], 'Lämpötilan keskiarvo [°C]': [6, 24], 'Pilvisyys [okta]': [6, 24] }
    print("Luodaan liukuvia tilastoja..."); rolling_cols_added = 0
    for col, windows in rolling_config.items():
         if col in original_cols:
            for window in windows:
                 df_eng[f'{col}_roll_mean_{window}h'] = df_eng[col].rolling(window=window, min_periods=1).mean()
                 df_eng[f'{col}_roll_std_{window}h'] = df_eng[col].rolling(window=window, min_periods=1).std()
                 rolling_cols_added += 2
         else: print(f"VAROITUS: Sarake '{col}' puuttuu tilastoja varten.")
    print(f"Tilastoja lisätty ({rolling_cols_added} kpl).")
    # Siivous
    initial_rows = len(df_eng); df_eng.dropna(inplace=True); rows_removed = initial_rows - len(df_eng)
    if rows_removed > 0: print(f"\nPoistettu {rows_removed} riviä alun NaN-arvojen vuoksi.")
    print(f"\nFE valmis. Lopullinen muoto: {df_eng.shape}"); return df_eng

def create_sequences_pytorch(features_scaled, targets_original, sequence_length, prediction_horizon):
    X, y_orig = [], []; print(f"\nLuodaan sekvenssejä: seq={sequence_length}, pred={prediction_horizon}")
    required_len = sequence_length + prediction_horizon
    if len(features_scaled) < required_len: print(f"VAROITUS: Ei dataa ({len(features_scaled)})."); return np.array(X), np.array(y_orig)
    if targets_original.ndim == 1: targets_original = targets_original.reshape(-1, 1)
    for i in range(len(features_scaled) - required_len + 1): X.append(features_scaled[i:(i + sequence_length)]); y_orig.append(targets_original[i + sequence_length : i + sequence_length + prediction_horizon, 0])
    print(f"Luotu {len(X)} sekvenssiä."); X = np.array(X); y_orig = np.array(y_orig)
    if y_orig.ndim == 2: y_orig = y_orig[..., np.newaxis]
    elif y_orig.size > 0: print(f"VAROITUS: y_orig muoto ({y_orig.shape}) ei odotettu.")
    return X, y_orig

print("--- Osa 2: Valmis (Funktiot määritelty) ---")

In [ ]:
# @title 2. Funktiot Datan Lataukseen ja Käsittelyyn (Laajennettu FE) (2025-04-10 19:44)

import pandas as pd
import numpy as np
import requests
import io
import os
import re
import traceback
# Varmistetaan tuonnit
try: from sklearn.preprocessing import StandardScaler
except ImportError: StandardScaler = None
try: import seaborn as sns; import matplotlib.pyplot as plt
except ImportError: sns = None; plt = None

# --- Funktiot datan lataamiseen ---
def download_data(url, local_path):
    try:
        print(f"Yritetään ladata: {url[:60]}..."); response = requests.get(url); response.raise_for_status()
        target_dir = os.path.dirname(local_path);
        if target_dir and not os.path.exists(target_dir): os.makedirs(target_dir, exist_ok=True)
        with open(local_path, 'wb') as f: f.write(response.content)
        print(f"Ladattu: {local_path}"); return True
    except Exception as e: print(f"Latausvirhe: {e}"); return False

def load_parquet_data(filepath_or_url, local_cache_path="default_cache.parquet"):
    filepath_to_read = None
    if filepath_or_url.startswith('http'):
         if os.path.exists(local_cache_path): filepath_to_read = local_cache_path; print(f"Käytetään välimuistia: {local_cache_path}")
         else: filepath_to_read = local_cache_path if download_data(filepath_or_url, local_cache_path) else None
    else: filepath_to_read = filepath_or_url
    if not filepath_to_read or not os.path.exists(filepath_to_read): print(f"VIRHE: Tiedostoa '{filepath_to_read}' ei löytynyt."); return None
    try:
        print(f"Ladataan: {filepath_to_read}"); df = pd.read_parquet(filepath_to_read); print(f"Parquet ladattu ({df.shape})")
        if not isinstance(df.index, pd.DatetimeIndex): df.index = pd.to_datetime(df.index)
        if df.index.tz is None: print("Asetetaan TZ=Europe/Helsinki..."); df = df.tz_localize('Europe/Helsinki', ambiguous='NaT', nonexistent='NaT')
        nat_rows = df.index.isnull();
        if nat_rows.any(): print(f"Poistetaan {nat_rows.sum()} NaT-riviä..."); df = df[~nat_rows]
        df.sort_index(inplace=True); print(f"Data aikaväliltä: [{df.index.min()} - {df.index.max()}]")
        if df.isnull().any().any(): print("NaN-arvoja. Täytetään..."); df=df.ffill().bfill(); df.dropna(inplace=True)
        print(f"Lopullinen ladattu muoto: {df.shape}"); return df
    except Exception as e: print(f"VIRHE luvussa/käsittelyssä: {e}"); traceback.print_exc(); return None

# --- Laajennettu Feature Engineering funktio ---
def feature_engineer_advanced_gru(df, target_column):
    print("\nSuoritetaan laajennettu FE..."); df_eng = df.copy()
    if not isinstance(df_eng.index, pd.DatetimeIndex): print("VIRHE: Indeksi ei DatetimeIndex."); return None
    try: # Aika
        df_eng['hour'] = df_eng.index.hour; df_eng['dayofweek'] = df_eng.index.dayofweek; df_eng['dayofyear'] = df_eng.index.dayofyear
        # Korjattu: Käytä df_eng.index.dayofyear (ei Series), ja days_in_year (Series)
        days_in_year = df_eng.index.is_leap_year.map({True: 366.0, False: 365.0})
        df_eng['hour_sin']=np.sin(2*np.pi*df_eng['hour']/24.0); df_eng['hour_cos']=np.cos(2*np.pi*df_eng['hour']/24.0)
        df_eng['dayofweek_sin']=np.sin(2*np.pi*df_eng['dayofweek']/7.0); df_eng['dayofweek_cos']=np.cos(2*np.pi*df_eng['dayofweek']/7.0)
        df_eng['dayofyear_sin']=np.sin(2*np.pi*df_eng['dayofyear']/days_in_year); df_eng['dayofyear_cos']=np.cos(2*np.pi*df_eng['dayofyear']/days_in_year)
        df_eng.drop(columns=['hour', 'dayofweek', 'dayofyear'], inplace=True); print("Lisätty sykliset aikaominaisuudet.")
    except Exception as e: print(f"VIRHE aika FE: {e}")
    wind_dir_col_orig = 'Tuulen suunnan keskiarvo [°]' # Tuuli
    if wind_dir_col_orig in df_eng.columns:
        try:
            wind_dir_numeric = pd.to_numeric(df_eng[wind_dir_col_orig], errors='coerce').ffill().bfill()
            if wind_dir_numeric.notna().all():
                 df_eng['wind_dir_sin']=np.sin(np.deg2rad(wind_dir_numeric)); df_eng['wind_dir_cos']=np.cos(np.deg2rad(wind_dir_numeric))
                 df_eng.drop(columns=[wind_dir_col_orig], inplace=True); print("Muunnettu tuulen suunta sykliseksi.")
            else: print(f"VAROITUS: Tuulensuuntia ei voitu muuntaa numeroksi.")
        except Exception as e: print(f"VIRHE tuulensuunta FE: {e}"); df_eng.drop(columns=['wind_dir_sin','wind_dir_cos'], inplace=True, errors='ignore')
    else: print(f"Saraketta '{wind_dir_col_orig}' ei löytynyt.")
    # Viiveet
    lag_config = { target_column: [1, 2, 3, 6, 12, 24, 48, 72, 168], 'Lämpötilan keskiarvo [°C]': [1, 6, 24], 'Pilvisyys [okta]': [1, 6, 24], 'Keskituulen nopeus [m/s]': [1, 6, 24] }
    print("Luodaan viiveitä..."); original_cols = list(df_eng.columns); lag_cols_added = 0
    for col, lags in lag_config.items():
        if col in original_cols:
            for lag in lags: df_eng[f'{col}_lag_{lag}h'] = df_eng[col].shift(lag); lag_cols_added+=1
        else: print(f"VAROITUS: Sarake '{col}' puuttuu viiveitä varten.")
    print(f"Viiveitä lisätty ({lag_cols_added} kpl).")
    # Liukuvat tilastot
    rolling_config = { target_column: [3, 6, 12, 24], 'Lämpötilan keskiarvo [°C]': [6, 24], 'Pilvisyys [okta]': [6, 24] }
    print("Luodaan liukuvia tilastoja..."); rolling_cols_added = 0
    for col, windows in rolling_config.items():
         if col in original_cols:
            for window in windows:
                 df_eng[f'{col}_roll_mean_{window}h'] = df_eng[col].rolling(window=window, min_periods=1).mean()
                 df_eng[f'{col}_roll_std_{window}h'] = df_eng[col].rolling(window=window, min_periods=1).std()
                 rolling_cols_added += 2
         else: print(f"VAROITUS: Sarake '{col}' puuttuu tilastoja varten.")
    print(f"Tilastoja lisätty ({rolling_cols_added} kpl).")
    # Siivous
    initial_rows = len(df_eng); df_eng.dropna(inplace=True); rows_removed = initial_rows - len(df_eng)
    if rows_removed > 0: print(f"\nPoistettu {rows_removed} riviä alun NaN-arvojen vuoksi.")
    print(f"\nFE valmis. Lopullinen muoto: {df_eng.shape}"); return df_eng

def create_sequences_pytorch(features_scaled, targets_original, sequence_length, prediction_horizon):
    X, y_orig = [], []; print(f"\nLuodaan sekvenssejä: seq={sequence_length}, pred={prediction_horizon}")
    required_len = sequence_length + prediction_horizon
    if len(features_scaled) < required_len: print(f"VAROITUS: Ei dataa ({len(features_scaled)})."); return np.array(X), np.array(y_orig)
    if targets_original.ndim == 1: targets_original = targets_original.reshape(-1, 1)
    for i in range(len(features_scaled) - required_len + 1): X.append(features_scaled[i:(i + sequence_length)]); y_orig.append(targets_original[i + sequence_length : i + sequence_length + prediction_horizon, 0])
    print(f"Luotu {len(X)} sekvenssiä."); X = np.array(X); y_orig = np.array(y_orig)
    if y_orig.ndim == 2: y_orig = y_orig[..., np.newaxis]
    elif y_orig.size > 0: print(f"VAROITUS: y_orig muoto ({y_orig.shape}) ei odotettu.")
    return X, y_orig

print("--- Osa 2: Valmis (Funktiot määritelty) ---")

In [ ]:
# @title 3. Pääskriptin Suoritus: Datan Käsittely (Laajennettu FE) (2025-04-10 19:44)

import pandas as pd
import numpy as np
import traceback
import seaborn as sns
import matplotlib.pyplot as plt
try: import torch; from torch.utils.data import TensorDataset, DataLoader
except ImportError: raise ImportError("PyTorch ei ladattu.")
try: from sklearn.preprocessing import StandardScaler
except ImportError: raise ImportError("scikit-learn ei ladattu.")

print("--- Aloitetaan Datan Käsittely (Laajennettu FE) ---")

# Alustetaan muuttujat
df_raw_full=None; df_engineered=None; train_loader=None; valid_loader=None; test_loader=None
feature_scaler=None; o3_scaler=None; INPUT_SIZE=None; test_timestamps=None
X_train, y_train_original, X_valid, y_valid_original, X_test, y_test_original = [None]*6
y_train_scaled, y_valid_scaled = None, None
data_processing_ok = False

try:
    # Varmistetaan parametrit uudelleen
    if 'SEQUENCE_LENGTH' not in locals(): SEQUENCE_LENGTH = 72
    if 'PREDICTION_HORIZON' not in locals(): PREDICTION_HORIZON = 24
    if 'TARGET_COLUMN' not in locals(): TARGET_COLUMN = 'Otsoni [µg/m³]'
    if 'BASE_COLUMNS_TO_LOAD' not in locals(): BASE_COLUMNS_TO_LOAD = ['Otsoni [µg/m³]', 'Lämpötilan keskiarvo [°C]', 'Keskituulen nopeus [m/s]', 'Ilmanpaineen keskiarvo [hPa]', 'Tuulen suunnan keskiarvo [°]', 'Pilvisyys [okta]']
    if 'TRAIN_HYPERPARAMS' not in locals() or 'batch_size' not in TRAIN_HYPERPARAMS: TRAIN_HYPERPARAMS = {'batch_size': 64}
    print(f"Käytetään: SeqLen={SEQUENCE_LENGTH}, PredHoriz={PREDICTION_HORIZON}, Batch={TRAIN_HYPERPARAMS['batch_size']}")

    # 1. Lataa data
    df_raw_full = load_parquet_data(DATA_URL, LOCAL_DATA_PATH)
    if df_raw_full is None or df_raw_full.empty: raise ValueError("Datan lataus epäonnistui.")

    # 2. Korrelaatioanalyysi
    print("\n--- Korrelaatioanalyysi ---")
    if sns is not None and plt is not None:
        try:
             numeric_cols_df = df_raw_full.select_dtypes(include=np.number)
             if TARGET_COLUMN in numeric_cols_df.columns:
                 correlation_matrix = numeric_cols_df.corr(); plt.figure(figsize=(8, 6))
                 sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", lw=.5, annot_kws={"size": 8})
                 plt.title('Muuttujien korrelaatiot', fontsize=12); plt.xticks(fontsize=8); plt.yticks(fontsize=8); plt.show()
                 print(f"\nKorrelaatiot '{TARGET_COLUMN}' kanssa:"); print(correlation_matrix[TARGET_COLUMN].sort_values(ascending=False))
             else: print(f"Kohdetta '{TARGET_COLUMN}' ei löytynyt.")
        except Exception as e_corr: print(f"VIRHE korrelaatioanalyysissä: {e_corr}")
    else: print("Ohitetaan korrelaatiomatriisi.")
    print("-" * 60)

    # 3. Laajennettu Feature Engineering
    df_engineered = feature_engineer_advanced_gru(df_raw_full.copy(), TARGET_COLUMN) # Käytä funktiota Osasta 2
    if df_engineered is None or df_engineered.empty: raise ValueError("Laajennettu FE epäonnistui.")

    # 4. Määritä INPUT_SIZE
    FINAL_FEATURE_COLUMNS = df_engineered.columns.tolist()
    if TARGET_COLUMN not in FINAL_FEATURE_COLUMNS: print(f"VAROITUS: Kohde '{TARGET_COLUMN}' ei FE:n jälkeisissä ominaisuuksissa!")
    INPUT_SIZE = len(FINAL_FEATURE_COLUMNS)
    if INPUT_SIZE == 0: raise ValueError("Ei lopullisia ominaisuuksia.");
    print(f"\nLopullinen ominaisuuksien määrä (INPUT_SIZE): {INPUT_SIZE}") # Tulostetaan koko

    # 5. Jaa data
    n = len(df_engineered); test_split_idx=int(n*(1-TEST_SPLIT_RATIO)); valid_split_idx=int(test_split_idx*(1-VALID_SPLIT_RATIO/(1-TEST_SPLIT_RATIO)))
    df_train=df_engineered[:valid_split_idx]; df_valid=df_engineered[valid_split_idx:test_split_idx]; df_test=df_engineered[test_split_idx:]
    min_len_for_seq=SEQUENCE_LENGTH+PREDICTION_HORIZON
    if len(df_train)<min_len_for_seq or len(df_valid)<min_len_for_seq or len(df_test)<min_len_for_seq: raise ValueError("Liian vähän dataa jaossa sekvensseille.")
    print(f"\nDatan jako (FE): Train={len(df_train)}, Valid={len(df_valid)}, Test={len(df_test)}")

    # 6. Skaalaa data
    print("\nSkaalataan..."); feature_scaler=StandardScaler(); scaled_train_features=feature_scaler.fit_transform(df_train)
    scaled_valid_features=feature_scaler.transform(df_valid); scaled_test_features=feature_scaler.transform(df_test)
    print("Ominaisuudet skaalattu.")
    o3_scaler = StandardScaler(); o3_scaler.fit(df_engineered.loc[df_train.index, [TARGET_COLUMN]]); print("O3 skaalain sovitettu.")

    # 7. Hae alkuperäiset O3-kohdearvot
    o3_train_targets_original=df_engineered.loc[df_train.index,[TARGET_COLUMN]].values
    o3_valid_targets_original=df_engineered.loc[df_valid.index,[TARGET_COLUMN]].values
    o3_test_targets_original=df_engineered.loc[df_test.index,[TARGET_COLUMN]].values

    # 8. Luo sekvenssit
    X_train,y_train_original=create_sequences_pytorch(scaled_train_features,o3_train_targets_original,SEQUENCE_LENGTH,PREDICTION_HORIZON)
    X_valid,y_valid_original=create_sequences_pytorch(scaled_valid_features,o3_valid_targets_original,SEQUENCE_LENGTH,PREDICTION_HORIZON)
    X_test,y_test_original=create_sequences_pytorch(scaled_test_features,o3_test_targets_original,SEQUENCE_LENGTH,PREDICTION_HORIZON)
    if X_train.size==0 or X_valid.size==0 or X_test.size==0: raise ValueError("Sekvenssien luonti epäonnistui.")

    # 9. Skaalaa kohdesekvenssit
    y_train_scaled=o3_scaler.transform(y_train_original.reshape(-1,1)).reshape(y_train_original.shape)
    y_valid_scaled=o3_scaler.transform(y_valid_original.reshape(-1,1)).reshape(y_valid_original.shape)
    print("Kohdesekvenssit skaalattu.")

    # 10. Muunna Tensoreiksi
    print("\nMuunnetaan Tensoreiksi..."); X_train_tensor=torch.tensor(X_train,dtype=torch.float32); y_train_tensor=torch.tensor(y_train_scaled,dtype=torch.float32)
    X_valid_tensor=torch.tensor(X_valid,dtype=torch.float32); y_valid_tensor=torch.tensor(y_valid_scaled,dtype=torch.float32)
    X_test_tensor=torch.tensor(X_test,dtype=torch.float32); y_test_tensor_original=torch.tensor(y_test_original,dtype=torch.float32)
    print("Tensorit luotu.")

    # 11. Luo DataLoaderit
    batch_size = TRAIN_HYPERPARAMS.get('batch_size', 64)
    train_dataset=TensorDataset(X_train_tensor,y_train_tensor); train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,drop_last=True)
    valid_dataset=TensorDataset(X_valid_tensor,y_valid_tensor); valid_loader=DataLoader(valid_dataset,batch_size=batch_size,shuffle=False)
    test_dataset=TensorDataset(X_test_tensor,y_test_tensor_original); test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
    print("DataLoaderit luotu.")

    # 12. Tallenna testiaikaleimat
    print("\nTallennetaan testiaikaleimoja...")
    if 'df_engineered' in locals() and not df_engineered.empty and 'df_test' in locals() and not df_test.empty and len(df_test)>=SEQUENCE_LENGTH and 'X_test' in locals() and len(X_test)>0:
        try:
            if df_test.index[0] in df_engineered.index:
                 test_start_idx_loc=df_engineered.index.get_loc(df_test.index[0]); test_start_indices=test_start_idx_loc+SEQUENCE_LENGTH; end_index=test_start_indices+len(X_test)
                 if end_index<=len(df_engineered.index): test_timestamps=df_engineered.index[test_start_indices:end_index]; print(f"Testiaikaleimat tallennettu ({len(test_timestamps)} kpl).")
                 else: print("VAROITUS: Ei voitu määrittää aikaleimoja."); test_timestamps=None
            else: print("VAROITUS: Testialkua ei löytynyt indeksistä."); test_timestamps=None
        except Exception as e_ts: print(f"VIRHE aikaleimoissa: {e_ts}"); test_timestamps=None
    else: print("VAROITUS: Data liian lyhyt/tyhjä aikaleimoille."); test_timestamps=None

    print(f"\nLopulliset muodot:"); print(f"X_train: {X_train_tensor.shape}, y_train: {y_train_tensor.shape}"); print(f"X_valid: {X_valid_tensor.shape}, y_valid: {y_valid_tensor.shape}"); print(f"X_test: {X_test_tensor.shape}, y_test: {y_test_tensor_original.shape}")
    data_processing_ok=True
except Exception as e: print(f"\n---> VIRHE DATAN KÄSITTELYSSÄ (Osa 3): {e} <---"); traceback.print_exc(); train_loader=None;INPUT_SIZE=None; data_processing_ok=False

# --- VAHVISTUSTULOSTE ---
if data_processing_ok: print("\n--- Osa 3: Valmis ---")
else: print("\n--- Osa 3: EPÄONNISTUI ---")

In [ ]:
# @title 4. GRU-Mallin Määrittely (GRU v4 - Laaj. FE) (2025-04-10 19:44)

import torch
import torch.nn as nn
import traceback

print("--- Määritellään GRU-malli ---")

model_definition_ok = False
# Tarkista edellisen osan onnistuminen ja INPUT_SIZE
if 'data_processing_ok' in locals() and data_processing_ok and 'INPUT_SIZE' in locals() and INPUT_SIZE is not None:
    try:
        input_size=INPUT_SIZE; hidden_size=RNN_HYPERPARAMS['hidden_size']; num_layers=RNN_HYPERPARAMS['num_layers']
        output_size=RNN_HYPERPARAMS['output_size']; dropout_prob=RNN_HYPERPARAMS['dropout_prob']
        print(f"GRU-parametrit: Input={input_size}, Hidden={hidden_size}, Layers={num_layers}, Output={output_size}, Dropout={dropout_prob}")
        assert all(isinstance(v, int) and v > 0 for v in [input_size, hidden_size, num_layers, output_size]), "Koot > 0"
        assert isinstance(dropout_prob, float) and 0.0 <= dropout_prob < 1.0, "Dropout [0, 1)"

        class GRUModel(nn.Module):
            def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
                super(GRUModel, self).__init__(); self.hidden_size=hidden_size; self.num_layers=num_layers
                gru_dropout = dropout_prob if num_layers > 1 else 0.0; self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=gru_dropout)
                self.fc = nn.Linear(hidden_size, output_size)
            def forward(self, x): h0=torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device); out, _ = self.gru(x, h0); out=out[:, -1, :]; out=self.fc(out); return out
        try: # Testataan alustus
             temp_model_test = GRUModel(input_size, hidden_size, num_layers, output_size, dropout_prob)
             print("GRUModel alustus OK."); print(temp_model_test); del temp_model_test; model_definition_ok = True
        except Exception as e_mdl: print(f"VIRHE GRUModel alustuksessa: {e_mdl}"); traceback.print_exc()
    except Exception as e_prm: print(f"VIRHE hyperparametreissa: {e_prm}")
else: print("Ohitetaan mallin määrittely (aiempi vaihe epäonnistui).")

# --- VAHVISTUSTULOSTE ---
if model_definition_ok: print("\n--- Osa 4: Valmis ---")
else: print("\n--- Osa 4: EPÄONNISTUI / OHITETTIIN ---")

In [ ]:
# @title 5. Koulutusfunktion Määrittely (GRU v4 - Laaj. FE) (2025-04-10 19:44)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm # Varmista tqdm tuonti
import copy
import traceback

print("--- Määritellään train_model -funktio ---")
training_function_ok = False
try:
    # Varmistetaan tarvittavat tuonnit tälle funktiolle
    if 'DataLoader' not in globals(): raise NameError("DataLoader puuttuu.")
    if 'nn' not in globals() or 'optim' not in globals(): raise NameError("PyTorch moduulit puuttuvat.")
    if 'tqdm' not in globals() or tqdm is None: print("Tqdm puuttuu, käytetään perustulostusta."); tqdm = lambda x, **kwargs: x # Korvike jos tqdm puuttuu

    def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs, device, patience):
        train_losses=[]; valid_losses=[]; best_valid_loss=float('inf'); epochs_no_improve=0; best_model_state=None
        print(f"\nAloitetaan koulutus {epochs} epochilla (patience={patience})...")
        if not train_loader or not valid_loader or len(train_loader.dataset)==0 or len(valid_loader.dataset)==0: print("VIRHE: Loader/Dataset tyhjä!"); return None,None,None
        for epoch in tqdm(range(epochs), desc="Epochs"):
            model.train(); running_train_loss=0.0; batch_count=0
            try:
                for inputs, targets_scaled in train_loader:
                    batch_count+=1; inputs, targets_scaled = inputs.to(device), targets_scaled.to(device)
                    optimizer.zero_grad(); outputs_scaled = model(inputs)
                    if targets_scaled.ndim==outputs_scaled.ndim+1 and targets_scaled.shape[-1]==1: targets_squeezed = targets_scaled.squeeze(-1)
                    elif targets_scaled.shape==outputs_scaled.shape: targets_squeezed=targets_scaled
                    else: raise RuntimeError(f"Muotovirhe (Train E{epoch+1} B{batch_count}): Out={outputs_scaled.shape}, Target={targets_scaled.shape}")
                    loss = criterion(outputs_scaled, targets_squeezed); loss.backward(); optimizer.step()
                    running_train_loss += loss.item() * inputs.size(0)
                denominator = len(train_loader.sampler) if hasattr(train_loader,'sampler') and train_loader.drop_last else len(train_loader.dataset)
                epoch_train_loss = running_train_loss / denominator if denominator > 0 else 0; train_losses.append(epoch_train_loss)
            except Exception as e_tr: print(f"\nVIRHE koulutusloopissa (E{epoch+1}): {e_tr}"); traceback.print_exc(); return None,train_losses,valid_losses
            model.eval(); running_valid_loss=0.0; valid_batch_count=0
            try:
                with torch.no_grad():
                    for inputs, targets_scaled in valid_loader:
                        valid_batch_count+=1; inputs,targets_scaled=inputs.to(device),targets_scaled.to(device); outputs_scaled=model(inputs)
                        if targets_scaled.ndim==outputs_scaled.ndim+1 and targets_scaled.shape[-1]==1: targets_squeezed=targets_scaled.squeeze(-1)
                        elif targets_scaled.shape==outputs_scaled.shape: targets_squeezed=targets_scaled
                        else: raise RuntimeError(f"Muotovirhe (Valid E{epoch+1} B{valid_batch_count}): Out={outputs_scaled.shape}, Target={targets_scaled.shape}")
                        loss=criterion(outputs_scaled,targets_squeezed); running_valid_loss+=loss.item()*inputs.size(0)
                epoch_valid_loss=running_valid_loss/len(valid_loader.dataset) if len(valid_loader.dataset)>0 else 0; valid_losses.append(epoch_valid_loss)
                print(f"Epoch {epoch+1:02d}/{epochs} Train:{epoch_train_loss:.5f} Valid:{epoch_valid_loss:.5f}", end="")
                if epoch_valid_loss<best_valid_loss:
                    best_valid_loss=epoch_valid_loss; epochs_no_improve=0
                    try: best_model_state=copy.deepcopy(model.state_dict()); print(" (Paras!)")
                    # *** Korjattu except-syntaksi ***
                    except Exception as e_save_state: print(f" (TILA EI TALLENNETTU: {e_save_state})"); best_model_state=None
                else: epochs_no_improve+=1; print(f" (Ei par.{epochs_no_improve}/{patience})")
                if epochs_no_improve>=patience: print("\nEarly stopping."); break
            except Exception as e_val: print(f"\nVIRHE validointiloopissa (E{epoch+1}): {e_val}"); traceback.print_exc(); return model,train_losses,valid_losses
        if best_model_state: print("\nLadataan paras malli."); model.load_state_dict(best_model_state)
        elif epochs>0 and train_losses is not None : print("\nEi Early Stoppingia. Käytetään viimeisintä.")
        else: print("\nKoulutusta ei ajettu / keskeytyi.")
        return model, train_losses, valid_losses
    training_function_ok = True
except NameError as ne: print(f"VIRHE: Tuonti puuttuu: {ne}")
except Exception as e_def: print(f"VIRHE train_model määrittelyssä: {e_def}"); traceback.print_exc()
# --- VAHVISTUSTULOSTE ---
if training_function_ok: print("\n--- Osa 5: Valmis (Funktio määritelty) ---")
else: print("\n--- Osa 5: EPÄONNISTUI ---")

In [ ]:
# @title 6. Mallin Koulutus (Suoritus) (GRU v4 - Laaj. FE) (2025-04-10 19:44)

import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import traceback

print("\n--- Osa 6: Mallin Koulutus (Suoritus) ---")
model = None; model_gru = None; train_losses = None; valid_losses = None; training_run_ok = False
# Tarkistetaan edellytykset huolellisemmin
prereqs_ok_for_training = ('data_processing_ok' in locals() and data_processing_ok and
                           'model_definition_ok' in locals() and model_definition_ok and
                           'training_function_ok' in locals() and training_function_ok and
                           'GRUModel' in locals() and GRUModel is not None and # Tarkista luokan olemassaolo
                           'train_model' in locals() and train_model is not None and # Tarkista funktion olemassaolo
                           'train_loader' in locals() and train_loader is not None and # Tarkista loaderit
                           'valid_loader' in locals() and valid_loader is not None and
                           'INPUT_SIZE' in locals() and INPUT_SIZE is not None) # Tarkista input size

if prereqs_ok_for_training:
    try:
        input_size=INPUT_SIZE; hidden_size=RNN_HYPERPARAMS['hidden_size']; num_layers=RNN_HYPERPARAMS['num_layers']
        output_size=RNN_HYPERPARAMS['output_size']; dropout_prob=RNN_HYPERPARAMS['dropout_prob']
        learning_rate=TRAIN_HYPERPARAMS['learning_rate']; epochs=TRAIN_HYPERPARAMS['epochs']; patience=TRAIN_HYPERPARAMS['patience']
        model=GRUModel(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device) # Käytä GRUModelia
        criterion=nn.MSELoss(); optimizer=optim.Adam(model.parameters(), lr=learning_rate)
        print(f"\n--- GRU-Malli (Input={input_size}), Params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
        # Kutsu koulutusfunktiota
        model, train_losses, valid_losses = train_model(model, train_loader, valid_loader, criterion, optimizer, epochs, device, patience)
        if model is not None and train_losses is not None and valid_losses is not None:
            print("\nKoulutus suoritettu."); model_gru=model; training_run_ok=True
            if train_losses and valid_losses:
                plt.figure(figsize=(10, 5)); plt.plot(train_losses, label='Training'); plt.plot(valid_losses, label='Validation')
                plt.title(f'Loss (GRU v4 - Laaj. FE)'); plt.xlabel('Epoch'); plt.ylabel('Loss (MSE)')
                try: min_loss=min(min(train_losses,default=1),min(valid_losses,default=1)); plt.yscale('log' if min_loss>1e-9 else 'linear')
                except: plt.yscale('linear')
                plt.legend(); plt.grid(True,alpha=0.6); plt.show()
            else: print("Häviölistoja ei saatu.")
        else: print("\nKoulutus epäonnistui.")
    except Exception as e_train_run: print(f"\nVIRHE koulutuksen ajossa: {e_train_run}"); traceback.print_exc(); model_gru=None
else: print(f"\nKoulutusta ei aloiteta, edellytykset puuttuvat.")
# --- VAHVISTUSTULOSTE ---
if training_run_ok: print("\n--- Osa 6: Valmis ---")
else: print("\n--- Osa 6: EPÄONNISTUI / OHITETTIIN ---")

In [ ]:
# @title 7. Arviointifunktioiden Määrittely (GRU v4 - Laaj. FE) (2025-04-10 19:51) - SyntaxError Korjattu

import numpy as np
import pandas as pd
import traceback
try:
    from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
except ImportError:
    raise ImportError("Sklearn puuttuu.")
try:
    from sklearn.preprocessing import StandardScaler
except ImportError:
    StandardScaler = None # Estää kaatumisen, mutta arviointi epäonnistuu myöhemmin jos tätä tarvitaan
    print("VAROITUS: StandardScaler tuonti epäonnistui!")
try:
    import torch
    from torch.utils.data import DataLoader
    from tqdm.notebook import tqdm
except ImportError:
    raise ImportError("PyTorch/tqdm puuttuu.")

print("--- Määritellään arviointifunktiot ---")
evaluation_functions_ok = False
try:
    # *** KORJATTU calculate_baseline_persistence_pytorch FUNKTIO ***
    def calculate_baseline_persistence_pytorch(targets_original_np, prediction_horizon):
        """Laskee naiivin persistenssi-baselinen PyTorch-datarakenteelle."""
        print("Lasketaan Baseline-ennuste (jakson eka arvo toistuu)...")
        # try: statement on nyt omalla rivillään ja seuraavat sisennetty
        try:
            if not isinstance(targets_original_np, np.ndarray): raise TypeError("Vaaditaan numpy array")
            # Tarkista muoto ja korjaa tarvittaessa
            if targets_original_np.ndim != 3 or targets_original_np.shape[-1] != 1:
                 if targets_original_np.ndim == 2 and targets_original_np.shape[1] == prediction_horizon:
                      print(f"Muutetaan baseline-syötteen muoto {targets_original_np.shape} -> 3D")
                      targets_original_np = targets_original_np[..., np.newaxis]
                 else:
                      raise ValueError(f"Odotettiin (samples, {prediction_horizon}, 1), saatiin {targets_original_np.shape}")

            if targets_original_np.shape[0] == 0:
                 print("Tyhjä syöte baselineen, palautetaan tyhjä.")
                 return np.zeros((0, prediction_horizon)) # Palauta oikea muoto

            # Ota ensimmäinen arvo ja toista
            first_vals = targets_original_np[:, 0, 0] # Muoto: (samples,)
            first_vals_for_repeat = first_vals[:, np.newaxis] # Muoto: (samples, 1)
            baseline_preds = np.repeat(first_vals_for_repeat, prediction_horizon, axis=1) # Muoto (samples, horizon)
            return baseline_preds
        except Exception as e_base:
            print(f"VIRHE baseline-laskennassa: {e_base}")
            # Yritetään palauttaa oikean muotoisia nollia
            try:
                # Oletetaan, että samples-määrä on ensimmäinen dimensio, jos se on olemassa
                num_samples = targets_original_np.shape[0] if hasattr(targets_original_np, 'shape') and len(targets_original_np.shape) > 0 else 0
                return np.zeros((num_samples, prediction_horizon))
            except:
                 return None # Viimeinen oljenkorsi

    def evaluate_model_performance_pytorch(model, model_name, test_loader, device, o3_scaler, o3_threshold_8h, prediction_horizon):
        """Arvioi PyTorch-mallia testidatalla, laskee metriikat ja vertaa baselineen."""
        print(f"\n--- evaluate_model_performance_pytorch ({model_name}) alkaa ---")
        if model is None: print("Malli puuttuu."); return None, None
        if test_loader is None: print("Testilataaja puuttuu."); return None, None
        if o3_scaler is None: print("O3-skaalain puuttuu."); return None, None
        # Varmistetaan skaalaimen tyyppi
        if StandardScaler is not None and not isinstance(o3_scaler, StandardScaler):
             print("VAROITUS: Annettu o3_scaler ei ole odotettua StandardScaler-tyyppiä.")

        model.eval(); all_preds_orig_list=[]; all_targets_orig_list=[]
        print("Ennustetaan testidatalla...");
        try:
            if not hasattr(test_loader, 'dataset') or len(test_loader.dataset) == 0:
                 print("Testidata on tyhjä."); return None,None
            with torch.no_grad():
                for inputs, targets_original_batch in tqdm(test_loader, desc=f"Testaus ({model_name})"):
                    inputs=inputs.to(device); outputs_scaled=model(inputs); preds_scaled_np=outputs_scaled.cpu().numpy()
                    preds_reshaped=preds_scaled_np.reshape(-1,1); preds_orig_np=o3_scaler.inverse_transform(preds_reshaped).reshape(preds_scaled_np.shape)
                    all_preds_orig_list.append(preds_orig_np); all_targets_orig_list.append(targets_original_batch.cpu().numpy())
            print("Ennusteet kerätty.");
            if not all_preds_orig_list or not all_targets_orig_list: raise ValueError("Listat tyhjiä.")
            all_preds_orig=np.concatenate(all_preds_orig_list,axis=0); all_targets_original=np.concatenate(all_targets_orig_list,axis=0)
            print(f"Kerätty {all_preds_orig.shape[0]} kpl.")
            if all_targets_original.ndim==3 and all_targets_original.shape[-1]==1: targets_eval=all_targets_original.squeeze(-1)
            elif all_targets_original.ndim==2: targets_eval=all_targets_original
            else: raise ValueError(f"Odottamaton kohdemuoto: {all_targets_original.shape}")
            if all_preds_orig.shape!=targets_eval.shape: raise ValueError(f"Muodot eivät täsmää: Pred={all_preds_orig.shape}, Target={targets_eval.shape}")
            print(f"Kohdemuoto {targets_eval.shape}")

            print("\nLasketaan metriikat..."); rmse_model=np.sqrt(mean_squared_error(targets_eval.ravel(),all_preds_orig.ravel())); mae_model=mean_absolute_error(targets_eval.ravel(),all_preds_orig.ravel())
            print(f"\n--- {model_name} Arviointi ---"); print(f"RMSE: {rmse_model:.4f}"); print(f"MAE:  {mae_model:.4f}")

            baseline_preds=calculate_baseline_persistence_pytorch(all_targets_original,prediction_horizon); rmse_baseline,mae_baseline=None,None
            if baseline_preds is not None and baseline_preds.shape==targets_eval.shape:
                if np.isnan(baseline_preds).any(): print("VAROITUS: Baseline NaN."); baseline_preds=pd.DataFrame(baseline_preds).ffill().bfill().values
                if not np.isnan(baseline_preds).any():
                    rmse_baseline=np.sqrt(mean_squared_error(targets_eval.ravel(),baseline_preds.ravel())); mae_baseline=mean_absolute_error(targets_eval.ravel(),baseline_preds.ravel())
                    print(f"\n--- Baseline Arviointi ---"); print(f"RMSE: {rmse_baseline:.4f}"); print(f"MAE:  {mae_baseline:.4f}")
                else: print("Baseline NaN.")
            else: print("Baseline epäonnistui / muoto väärä.")

            print("\n--- Vertailu Baselineen ---")
            if rmse_model is not None and rmse_baseline is not None: print(f"{model_name} vs Baseline RMSE: {rmse_baseline - rmse_model:+.4f} ({'Malli parempi' if (rmse_baseline - rmse_model) > 0 else 'Baseline parempi/sama'})")
            else: print("RMSE-vertailua ei voida tehdä.")
            if mae_model is not None and mae_baseline is not None: print(f"{model_name} vs Baseline MAE:  {mae_baseline - mae_model:+.4f} ({'Malli parempi' if (mae_baseline - mae_model) > 0 else 'Baseline parempi/sama'})")
            else: print("MAE-vertailua ei voida tehdä.")

            warnings_actual=[]; warnings_pred=[]; n_samples_total=all_preds_orig.shape[0]
            print(f"\nLasketaan 8h ka ({n_samples_total} jaksoa, kynnys={o3_threshold_8h})...");
            for i in range(n_samples_total):
                actual_series=pd.Series(targets_eval[i,:]); pred_series=pd.Series(all_preds_orig[i,:])
                actual_8h_avg=actual_series.rolling(window=8,min_periods=1).mean(); pred_8h_avg=pred_series.rolling(window=8,min_periods=1).mean()
                warnings_actual.append(actual_8h_avg.max()>o3_threshold_8h); warnings_pred.append(pred_8h_avg.max()>o3_threshold_8h)
            warnings_actual=np.array(warnings_actual); warnings_pred=np.array(warnings_pred); print("Laskenta valmis.")
            print(f"\n--- 8h Varoitus Arviointi ({model_name} - Jaksoittain) ---"); print(f"Todellisia (> {o3_threshold_8h}): {warnings_actual.sum()}/{n_samples_total}"); print(f"Ennustettuja ({model_name}): {warnings_pred.sum()}/{n_samples_total}")
            if warnings_actual.sum()>0 or warnings_pred.sum()>0:
                print("\nSekaannusmatriisi:"); cm=confusion_matrix(warnings_actual,warnings_pred,labels=[False,True]); print(pd.DataFrame(cm, index=['Todellinen EI','Todellinen KYLLÄ'], columns=['Ennuste EI','Ennuste KYLLÄ']))
                print("\nLuokitteluraportti:"); report=classification_report(warnings_actual,warnings_pred,target_names=['Ei Varoitusta','Varoitus'],labels=[False,True],zero_division=0); print(report)
                TN,FP,FN,TP=cm.ravel(); recall=TP/(TP+FN) if (TP+FN)>0 else 0; precision=TP/(TP+FP) if (TP+FP)>0 else 0; print(f"---> Recall: {recall:.4f}, Precision: {precision:.4f}")
            else: print(f"Ei ylityksiä kynnysarvolla {o3_threshold_8h}.")
            print(f"\n--- evaluate_model_performance_pytorch ({model_name}) päättyi ---"); return all_preds_orig, targets_eval
        except Exception as e: print(f"\n-----> VIRHE evaluate_model_performance_pytorch <-----"); traceback.print_exc(); return None,None
    evaluation_functions_ok = True
except NameError as ne: print(f"VIRHE: Tuonti puuttuu: {ne}")
except Exception as e_def: print(f"VIRHE arviointifunktioiden määrittelyssä: {e_def}"); traceback.print_exc()
# --- VAHVISTUSTULOSTE ---
if evaluation_functions_ok: print("\n--- Osa 7: Valmis (Funktiot määritelty) ---")
else: print("\n--- Osa 7: EPÄONNISTUI ---")

In [ ]:
# @title 8. Mallin Arviointi (Suoritus) (GRU v4 - Laaj. FE) (2025-04-10 19:44)

import traceback
import numpy as np
import pandas as pd

print("\n--- Osa 8: Mallin Arviointi (Suoritus) ---")
test_preds_gru=None; test_targets_gru=None; evaluation_gru_successful=False
# Tarkista kaikki edellytykset huolellisesti
prereqs_ok_for_eval = ('evaluation_functions_ok' in locals() and evaluation_functions_ok and
                       'model_gru' in locals() and model_gru is not None and
                       'test_loader' in locals() and test_loader is not None and
                       'o3_scaler' in locals() and o3_scaler is not None and 'device' in locals() and
                       'O3_THRESHOLD_8H_AVG' in locals() and 'PREDICTION_HORIZON' in locals() and
                       'evaluate_model_performance_pytorch' in locals() and evaluate_model_performance_pytorch is not None)

if prereqs_ok_for_eval:
    try:
        print(f"\nKutsutaan evaluate_model_performance_pytorch GRU-mallille (kynnys={O3_THRESHOLD_8H_AVG})...")
        test_preds_gru, test_targets_gru = evaluate_model_performance_pytorch(
            model_gru, "GRU (Laaj. FE)", test_loader, device, o3_scaler, O3_THRESHOLD_8H_AVG, PREDICTION_HORIZON
        )
        if test_preds_gru is not None and test_targets_gru is not None:
            print("Arviointifunktion ajo suoritettu."); evaluation_gru_successful = True
            print(f"Tulosten muodot: Preds={test_preds_gru.shape}, Targets={test_targets_gru.shape}")
        else: print("\nArviointi epäonnistui (funktio palautti None).")
    except Exception as e: print(f"\nVIRHE arvioinnin suorituksessa: {e}"); traceback.print_exc()
else: print(f"\nArviointia ei voida suorittaa, edellytykset puuttuvat.")
# --- VAHVISTUSTULOSTE ---
if evaluation_gru_successful: print("\n--- Osa 8: Valmis ---")
else: print("\n--- Osa 8: EPÄONNISTUI / OHITETTIIN ---")

In [ ]:
# @title 9. Viimeisimmän Ennustejakson Huippuarvo (GRU v4 - Laaj. FE) (2025-04-10 20:14) - NameError Korjattu

import numpy as np
import pandas as pd
import traceback

print("\n--- Osa 9: Viimeisimmän Ennustejakson Huippuarvo ---")
latest_analysis_possible = False

# Tarkista edellytykset (arvioinnin onnistuminen ja tulosmuuttujat)
# Käytetään Osassa 8 määriteltyjä/tallennettuja tuloksia
if 'evaluation_gru_successful' in locals() and evaluation_gru_successful and \
   'test_preds_gru' in locals() and isinstance(test_preds_gru, np.ndarray) and \
   'test_timestamps' in locals() and test_timestamps is not None:

    # Varmistetaan, ettei data ole tyhjää ja pituudet täsmäävät
    if len(test_preds_gru) > 0 and len(test_timestamps) == len(test_preds_gru):
        latest_analysis_possible = True
        try:
            latest_prediction_sequence = test_preds_gru[-1] # Viimeinen 24h jakso
            prediction_start_time = test_timestamps[-1]     # Viimeisen jakson alkuaika

            # Lasketaan max_idx ja max_val
            max_idx = np.argmax(latest_prediction_sequence) # Oikea nimi on max_idx
            max_value = latest_prediction_sequence[max_idx]

            # *** KORJAUS TÄSSÄ: Käytetään max_idx eikä max_index ***
            time_of_max = prediction_start_time + pd.Timedelta(hours=int(max_idx))

            print(f"\nViimeisin ennustettu 24h jakso alkaa: {prediction_start_time.strftime('%Y-%m-%d %H:%M')}")
            print("-" * 40);
            print(f"Korkein ennustettu O3-arvo: {max_value:.2f} µg/m³")
            print(f"Ajankohta: {time_of_max.strftime('%A, %d.%m.%Y klo %H:%M')}") # Tulostaa esim. Maanantai, 01.01.2024 klo 15:00
            print("-" * 40)

        except IndexError:
            print("\nVirhe indeksissä (todennäköisesti tyhjä data)."); latest_analysis_possible = False
        except Exception as e:
            print(f"\nVirhe viimeisimmän jakson analyysissa: {e}"); traceback.print_exc(); latest_analysis_possible = False
    else:
        print("\nEi voitu analysoida: Ennusteiden tai aikaleimojen pituudet eivät täsmää tai data puuttuu.")
else:
    # Tulostetaan selkeämmin, miksi ei voida analysoida
    missing_vars_for_peak = []
    if 'evaluation_gru_successful' not in locals() or not evaluation_gru_successful: missing_vars_for_peak.append('evaluation_gru_successful (False)')
    if 'test_preds_gru' not in locals() or not isinstance(test_preds_gru, np.ndarray): missing_vars_for_peak.append('test_preds_gru')
    if 'test_timestamps' not in locals() or test_timestamps is None: missing_vars_for_peak.append('test_timestamps')
    print(f"\nEi voitu analysoida huippuarvoa, koska arvioinnin tuloksia tai edellytyksiä puuttuu: {missing_vars_for_peak}")

# --- VAHVISTUSTULOSTE ---
if latest_analysis_possible:
    print("\n--- Osa 9: Valmis ---")
else:
    print("\n--- Osa 9: EPÄONNISTUI / OHITETTIIN ---")

In [ ]:
# @title 10. Visualisointi (Suoritus) (GRU v4 - Laaj. FE) (2025-04-10 20:44) # Päivitetty aika

import matplotlib.pyplot as plt # Varmista tuonti
import numpy as np # Varmista tuonti
import pandas as pd # Varmista tuonti
import traceback
import time # Tuodaan ajan laskentaa varten

print("\n--- Osa 10: Visualisointi (Suoritus) ---")
visualization_possible = False # Lipuke

# Tarkista edellytykset: Arvioinnin onnistuminen ja tulosmuuttujien olemassaolo
# Käytetään _gru päätteisiä muuttujia
if 'evaluation_gru_successful' in locals() and evaluation_gru_successful and \
   'test_preds_gru' in locals() and isinstance(test_preds_gru, np.ndarray) and \
   'test_targets_gru' in locals() and isinstance(test_targets_gru, np.ndarray) and \
   'test_timestamps' in locals() and test_timestamps is not None:

    print("\nPiirretään kuvaajat...")
    num_test_samples = len(test_preds_gru)
    # Varmistetaan pituudet
    lengths_ok = (num_test_samples > 0 and
                  test_targets_gru.shape[0] == num_test_samples and
                  len(test_timestamps) == num_test_samples)

    if lengths_ok:
        visualization_possible = True # Merkitään onnistuneeksi tähän asti
        # Valitaan satunnainen indeksi piirrettäväksi
        sample_idx = np.random.randint(0, num_test_samples)
        print(f"\nNäytejakso #{sample_idx}...")

        try: # --- Kuvaaja 1: Esimerkkijakson ennuste vs. Todellinen ---
            plt.figure(figsize=(16, 7))
            start_time = test_timestamps[sample_idx]
            # Varmistetaan PREDICTION_HORIZON olemassaolo
            if 'PREDICTION_HORIZON' not in locals(): PREDICTION_HORIZON=24 # Oletusarvo
            time_axis = pd.date_range(start=start_time, periods=PREDICTION_HORIZON, freq='h')

            targets_plot = test_targets_gru[sample_idx, :] # Pitäisi olla 1D (horizon,)
            if targets_plot.ndim != 1:
                 # Yritetään korjata, jos muoto on (horizon, 1)
                 if targets_plot.ndim == 2 and targets_plot.shape[-1] == 1:
                      targets_plot = targets_plot.squeeze(-1)
                 else:
                      raise ValueError(f"targets_plot muoto {targets_plot.shape}, odotettiin 1D")

            plt.plot(time_axis, targets_plot, label='Todellinen O3', marker='.', alpha=0.8, color='royalblue', zorder=3)
            plt.plot(time_axis, test_preds_gru[sample_idx, :], label='Ennuste O3 (GRU Laaj. FE)', marker='.', linestyle='--', alpha=0.8, color='darkorange', zorder=4)

            try: # Liukuvat keskiarvot
                actual_series = pd.Series(targets_plot, index=time_axis)
                pred_series = pd.Series(test_preds_gru[sample_idx, :], index=time_axis)
                actual_8h = actual_series.rolling(window=8, min_periods=1).mean()
                pred_8h = pred_series.rolling(window=8, min_periods=1).mean()
                plt.plot(time_axis, actual_8h, label='Todellinen 8h ka.', color='blue', ls='-', lw=2.5, alpha=0.5, zorder=1)
                plt.plot(time_axis, pred_8h, label='Ennustettu 8h ka.', color='red', ls=':', lw=2.5, alpha=0.5, zorder=2)
            except Exception as e_roll:
                 print(f"VAROITUS: Virhe 8h ka piirrossa: {e_roll}")

            # Varmistetaan O3_THRESHOLD_8H_AVG olemassaolo
            if 'O3_THRESHOLD_8H_AVG' not in locals(): O3_THRESHOLD_8H_AVG = 85 # Oletusarvo
            plt.axhline(O3_THRESHOLD_8H_AVG, color='crimson', ls='-.', lw=2, label=f'8h Kynnys ({O3_THRESHOLD_8H_AVG})', zorder=5)
            plt.title(f'GRU (Laaj. FE) Ennuste vs Todellinen #{sample_idx} ({start_time.strftime("%Y-%m-%d %H:%M")})', fontsize=14)
            plt.xlabel('Aika', fontsize=12); plt.ylabel('O3 (µg/m³)', fontsize=12)
            plt.legend(loc='best', fontsize=10); plt.grid(True, linestyle=':', alpha=0.6); plt.xticks(rotation=30, ha='right'); plt.tight_layout(); plt.show()

        except Exception as e_fig1:
            print(f"VIRHE Kuvaaja 1: {e_fig1}"); traceback.print_exc(); visualization_possible = False

        try: # --- Kuvaaja 2: Hajontakuvaaja (t+1) ---
             plt.figure(figsize=(7, 7));
             preds_t1 = test_preds_gru[:, 0]; targets_t1 = test_targets_gru[:, 0] # Pitäisi olla 1D (samples,)
             if targets_t1.ndim != 1 or preds_t1.ndim != 1: raise ValueError("Datamuodot väärin hajontakuvaajalle.")

             plt.scatter(targets_t1, preds_t1, alpha=0.3, label='Ennusteet (t+1)', s=20, edgecolors='k', lw=0.5)
             # NaN/Inf tarkistukset
             valid_targets=targets_t1[~np.isnan(targets_t1)&~np.isinf(targets_t1)]; valid_preds=preds_t1[~np.isnan(preds_t1)&~np.isinf(preds_t1)]
             if len(valid_targets)>0 and len(valid_preds)>0:
                  min_val=min(valid_targets.min(),valid_preds.min())-5; max_val=max(valid_targets.max(),valid_preds.max())+5
                  if np.isfinite(min_val) and np.isfinite(max_val) and max_val>min_val:
                      plt.plot([min_val,max_val],[min_val,max_val],color='red',ls='--',lw=2,label='Ihanteellinen (y=x)')
                      plt.xlim(min_val,max_val); plt.ylim(min_val,max_val)
                  else: print("VAROITUS: Ei voitu asettaa hajontakuvaajan rajoja.")
             else: print("VAROITUS: Ei validia dataa hajontakuvaajan rajoille.")

             plt.title('Hajontakuvaaja: Ennuste vs Todellinen (tunti t+1)', fontsize=14); plt.xlabel('Todellinen O3 (t+1)'); plt.ylabel('Ennustettu O3 (t+1)')
             plt.grid(True, ls=':', alpha=0.6); plt.legend(fontsize=10); plt.gca().set_aspect('equal', adjustable='box'); plt.tight_layout(); plt.show()

        except Exception as e_fig2:
             print(f"VIRHE Kuvaaja 2: {e_fig2}"); traceback.print_exc(); visualization_possible = False
    else:
        # Jos pituudet eivät täsmänneet
        print("Testidataa ei löytynyt tai pituudet eivät täsmää visualisointia varten.")
        visualization_possible = False

else:
    # Jos arvioinnin tuloksia tai aikaleimoja puuttui
    missing_vars_vis = []
    if 'evaluation_gru_successful' not in locals() or not evaluation_gru_successful: missing_vars_vis.append('evaluation_gru_successful')
    if 'test_preds_gru' not in locals() or not isinstance(test_preds_gru, np.ndarray): missing_vars_vis.append('test_preds_gru')
    if 'test_targets_gru' not in locals() or not isinstance(test_targets_gru, np.ndarray): missing_vars_vis.append('test_targets_gru')
    if 'test_timestamps' not in locals() or test_timestamps is None: missing_vars_vis.append('test_timestamps')
    print(f"\nVisualisointia ei suoriteta puuttuvien edellytysten vuoksi: {missing_vars_vis}")
    visualization_possible = False


# --- VAHVISTUSTULOSTE ---
if visualization_possible:
    print("\n--- Osa 10: Valmis ---")
else:
    print("\n--- Osa 10: EPÄONNISTUI / OHITETTIIN ---")

# --- Ajanotto (korjattu time importilla) ---
print(f"\n--- Koko skriptin ajo päättyi ---")
script_end_time = time.time() # Nyt time on tuotu
# Tarkistetaan onko script_start_time määritelty (pitäisi olla jos Osa 1 ajettu)
if 'script_start_time' in locals() and script_start_time is not None:
     try: print(f"Kokonaisajoaika: {script_end_time - script_start_time:.2f} sekuntia.")
     except Exception as e_time: print(f"Ajoajan laskenta epäonnistui: {e_time}")
else: print("Kokonaisajoaikaa ei voitu laskea (script_start_time puuttui).")